<a href="https://colab.research.google.com/github/Bainer13/Data_BBVA_Team_1/blob/Rodri/Proyecto_bootcamp_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Armado del entorno de trabajo



In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [2]:
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz

In [3]:
ls

address_bootcamp.snappy.parquet          phones_bootcamp.snappy.parquet
customer_basics_bootcamp.snappy.parquet  sample_data/
emails_bootcamp.snappy.parquet           spark-3.3.2-bin-hadoop3.tgz


In [4]:
!tar xf spark-3.3.2-bin-hadoop3.tgz

In [5]:
!java -version

openjdk version "11.0.18" 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)


In [6]:
!readlink -f $(which java) | sed "s:bin/java::"


/usr/lib/jvm/java-11-openjdk-amd64/


In [7]:
!echo $JAVA_HOME

In [8]:
!pip install -q findspark

In [9]:
!pip install pyspark 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 18.1 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=0bede9f4aa81b2a6dc12cde9c6f1ee600298abf2c9454c133ebccc268cb550e5
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [10]:
from pyspark.sql import SparkSession

In [11]:
import pyspark.sql.functions as F

In [12]:
import pandas as pd
pd.set_option('display.max_rows', None)

In [158]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import col
from pyspark.sql.functions import *

In [14]:
spark = SparkSession.builder.getOrCreate()

In [15]:
spark.conf.set('spark.sql.legacy.parquet.datetimeRebaseModeInRead', 'CORRECTED')

#Extraccion de datos

In [267]:
#Lectura de parquets
df_address = spark.read.parquet('/content/address_bootcamp.snappy.parquet')
df_customer_basics = spark.read.parquet('/content/customer_basics_bootcamp.snappy.parquet')
df_email = spark.read.parquet('/content/emails_bootcamp.snappy.parquet')
df_phones = spark.read.parquet('/content/phones_bootcamp.snappy.parquet')

**HAGO LOS JOIN DE LA TABLA DE CLIENTES CON LAS TABLAS DE DATOS POR SEPARADO**

In [ ]:
#Join entre clientes y telefonos
df_customer_basics_phones = df_customer_basics.join(df_phones, ['customer_id'], how='left').select(df_customer_basics.customer_id, df_phones.prefix_phone_id, df_phones.phone_area_id, df_phones.cellphone_prefix_id, df_phones.phone_exchange_id, df_phones.phone_line_id, df_phones.phone_id, df_phones.last_change_date)
df_customer_basics_phones.sort(F.asc("customer_id")).show(10)

In [ ]:
#Join entre clientes y email
df_customer_basics_email = df_customer_basics.join(df_email,['customer_id'], how='left').select(df_customer_basics.customer_id, df_email.email_desc, df_email.last_change_date)
df_customer_basics_email.show()

In [ ]:
#join entre clientes y direccion
df_customer_basics_address = df_customer_basics.join(df_address,['customer_id'], how='left').select(df_customer_basics.customer_id, 'street_name', 'address_outdoor_id','address_indoor_id', 'indoor_number', 'address_town_name', 'address_department_name', 'zipcode_id', df_address.last_change_date)
df_customer_basics_address.show()

**FILTRADO Y REDUCCION DE DATOS --> PHONES**

In [272]:
#Hago una ventana por el customer_id y ordenado por la ultima actualizacion 
w = Window.partitionBy(df_customer_basics_phones['customer_id']).orderBy("last_change_date")

In [273]:
#Agrego columna que cuente la cantidad de veces que se repite el ID
df_customer_basics_phones = df_customer_basics_phones.withColumn("row_num", row_number().over(w))

In [274]:
#Corto el dataframe quedandome con las ultimas 3 actualizaciones
df_customer_basics_phones = df_customer_basics_phones.filter(col("row_num") <= 3)

In [ ]:
#Reacomodo la tabla con las columnas de interes y concatenando una nueva
df_customer_basics_phones = df_customer_basics_phones.withColumn('phone_number', F.concat('phone_exchange_id', 'phone_line_id'))
df_customer_basics_phones = df_customer_basics_phones.select('customer_id', 'prefix_phone_id', 'phone_area_id', 'phone_number', 'row_num')
df_customer_basics_phones.show(10)

In [276]:
#creo una funcion pivot para una columna especificada y cambiarle el nombre a las nuevas.
def pivote(a):   
    df_pivote = df_customer_basics_phones.groupBy("customer_id").pivot("row_num").agg(first(a))
    df_pivote = df_pivote.withColumnRenamed("1", a + "_1") \
       .withColumnRenamed("2", a + "_2") \
       .withColumnRenamed("3", a + "_3")
    return df_pivote 

In [277]:
df_pivot = pivote('prefix_phone_id')
df_pivot2 = pivote('phone_area_id')
df_pivot3 = pivote('phone_number')

In [278]:
#Uno las tres tablas pivot y reacomodo las columnas.
joined_df = df_pivot.join(df_pivot2, on='customer_id', how='outer')
df_final_customer_phones = joined_df.join(df_pivot3, on='customer_id', how='outer')
df_final_customer_phones = df_final_customer_phones.select('customer_id', 'prefix_phone_id_1', 'phone_area_id_1', 'phone_number_1', 'prefix_phone_id_2', 'phone_area_id_2', 'phone_number_2', 'prefix_phone_id_3', 'phone_area_id_3', 'phone_number_3')
df_final_customer_phones.show()

+-----------+-----------------+---------------+--------------+-----------------+---------------+--------------+-----------------+---------------+--------------+
|customer_id|prefix_phone_id_1|phone_area_id_1|phone_number_1|prefix_phone_id_2|phone_area_id_2|phone_number_2|prefix_phone_id_3|phone_area_id_3|phone_number_3|
+-----------+-----------------+---------------+--------------+-----------------+---------------+--------------+-----------------+---------------+--------------+
|   00000442|               54|            297|       5296284|               54|            297|       4729337|             null|           null|          null|
|   00001419|               54|             11|      66793207|             null|           null|          null|             null|           null|          null|
|   00001939|               54|             11|      38700150|             null|           null|          null|             null|           null|          null|
|   00002575|             null|   

**FILTRADO Y REDUCCION DE DATOS --> EMAILS**

In [279]:
#Filtrado y reduccion de datos --> EMAILS
w = Window.partitionBy(df_customer_basics_email['customer_id']).orderBy("last_change_date")

In [280]:
df_customer_basics_email = df_customer_basics_email.withColumn("row_num", row_number().over(w))

In [281]:
df_customer_basics_email = df_customer_basics_email.filter(col("row_num") <= 3)

In [ ]:
df_customer_basics_email.show(10)

In [ ]:
#Hago primer tabla pivot para dejar una fila por ID y tener cada email en columnas separadas
df_final_customer_email = df_customer_basics_email.groupBy("customer_id").pivot("row_num").agg(first("email_desc"))
df_final_customer_email = df_final_customer_email.withColumnRenamed("1", "email_1") \
       .withColumnRenamed("2", "email_2") \
       .withColumnRenamed("3", "email_3")
df_final_customer_email.show()

**Filtrado y reduccion de datos --> ADDRESS**

In [284]:
#Hago una ventana por el customer_id y ordenado por la ultima actualizacion 
w = Window.partitionBy(df_customer_basics_address['customer_id']).orderBy("last_change_date")

In [285]:
#Agrego columna que cuente la cantidad de veces que se repite el ID
df_customer_basics_address = df_customer_basics_address.withColumn("row_num", row_number().over(w))

In [286]:
#Corto el dataframe quedandome con las ultimas 3 actualizaciones
df_customer_basics_address = df_customer_basics_address.filter(col("row_num") <= 3)

In [ ]:
#Reacomodo la tabla con las columnas de interes.
df_customer_basics_address = df_customer_basics_address.select('customer_id', 'street_name', 'address_outdoor_id', 'address_indoor_id', 'indoor_number', 'address_town_name', 'address_department_name', 'zipcode_id', 'row_num')
df_customer_basics_address.show(10)

In [288]:
#creo una funcion pivot para una columna especificada y cambiarle el nombre a las nuevas.
def pivote2(a):   
    df_pivote = df_customer_basics_address.groupBy("customer_id").pivot("row_num").agg(first(a))
    df_pivote = df_pivote.withColumnRenamed("1", a + "_1") \
       .withColumnRenamed("2", a + "_2") \
       .withColumnRenamed("3", a + "_3")
    return df_pivote 

In [289]:
#Aplico la funcion para cada una de las columnas de la tabla de customers-address
df_pivot4 = pivote2('street_name')
df_pivot5 = pivote2('address_outdoor_id')
df_pivot6 = pivote2('address_indoor_id')
df_pivot7 = pivote2('indoor_number')
df_pivot8 = pivote2('address_town_name')
df_pivot9 = pivote2('address_department_name')
df_pivot10 = pivote2('zipcode_id')

In [290]:
#Uno las tres tablas pivot y reacomodo las columnas.
join1 = df_pivot4.join(df_pivot5, on='customer_id', how='outer')
join2 = join1.join(df_pivot6, on='customer_id', how='outer')
join3 = join2.join(df_pivot7, on='customer_id', how='outer')
join4 = join3.join(df_pivot8, on='customer_id', how='outer')
join5 = join4.join(df_pivot9, on='customer_id', how='outer')
join6 = join5.join(df_pivot10, on='customer_id', how='outer')
df_final_customer_address = join6.select('customer_id', 'street_name_1', 'address_outdoor_id_1', 'address_indoor_id_1', 'indoor_number_1', 'address_department_name_1', 'zipcode_id_1', 'street_name_2', 'address_outdoor_id_2', 'address_indoor_id_2', 'indoor_number_2', 'address_department_name_2', 'zipcode_id_2', 'street_name_3', 'address_outdoor_id_3', 'address_indoor_id_3', 'indoor_number_3', 'address_department_name_3', 'zipcode_id_3')
df_final_customer_address.show()

+-----------+--------------------+--------------------+-------------------+---------------+-------------------------+------------+-------------+--------------------+-------------------+---------------+-------------------------+------------+-------------+--------------------+-------------------+---------------+-------------------------+------------+
|customer_id|       street_name_1|address_outdoor_id_1|address_indoor_id_1|indoor_number_1|address_department_name_1|zipcode_id_1|street_name_2|address_outdoor_id_2|address_indoor_id_2|indoor_number_2|address_department_name_2|zipcode_id_2|street_name_3|address_outdoor_id_3|address_indoor_id_3|indoor_number_3|address_department_name_3|zipcode_id_3|
+-----------+--------------------+--------------------+-------------------+---------------+-------------------------+------------+-------------+--------------------+-------------------+---------------+-------------------------+------------+-------------+--------------------+-------------------+---